In [2]:
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from pathlib import Path
from pprint import pprint

In [12]:
env_path = Path.home() / 'prod-bot' / '.env'
if not env_path.exists():
    print(f"Environment file not found at {env_path}")
else:
    load_dotenv(env_path)

In [23]:
def get_shares_overview(api_key, function, symbol):
    url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"
    return requests.get(url).json()


In [ ]:
api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
print(api_key)

In [ ]:
overview = get_shares_overview(api_key, "ETF_PROFILE", "IBIT")
pprint(overview)

In [25]:
def get_shares_outstanding(api_key, function, symbol):
    response = get_shares_overview(api_key, function, symbol)
    print(response)
    if shares_outstanding := response.get("SharesOutstanding"):
        return int(shares_outstanding)
    else:
        raise ValueError("Couldn't find Shares Outstanding data.")

In [ ]:
shares_outstanding = get_shares_outstanding(api_key, "ETF_PROFILE", "QQQ")
print(shares_outstanding)

In [ ]:
def get_ibit_btc_holdings():
    url = "https://treasuries.bitbo.io/blackrock-ibit/"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract total BTC holdings
    btc_amount_elem = soup.select_one('h2.text-4xl.font-bold')
    if btc_amount_elem:
        btc_held = btc_amount_elem.text.strip().replace(',', '')
        return float(btc_held)
    else:
        raise ValueError("Couldn't find BTC holdings data.")

In [ ]:
h = get_ibit_btc_holdings()

print(h)

To get IBIT Shares Outstanding:
1. Go to the IBIT BlackRock product page: https://www.blackrock.com/us/individual/products/333011/
2. Data Download: downloads an `.xls` file
3. In that file is a section
```xml
<ss:Cell>
<ss:Data ss:Type="String">Shares Outstanding</ss:Data>
</ss:Cell>
<ss:Cell ss:StyleID="Right">
<ss:Data ss:Type="String">999,200,000.00</ss:Data>
</ss:Cell>
```
4. Extract the data and convert to integer - this is the Shares Outstanding 

In [7]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import csv

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Step 1: Fetch the product page
product_url = "https://www.ishares.com/us/products/333011/ishares-bitcoin-trust"
response = requests.get(product_url, headers=headers)
if response.status_code != 200:
    print(f"Failed to fetch product page. Status code: {response.status_code}")

In [3]:

# Step 2: Parse HTML and find the holdings CSV link
soup = BeautifulSoup(response.text, 'html.parser')
holdings_link = None

# Look for the specific link by class or text
for a in soup.find_all('a', href=True):
    if 'icon-xls-export' in a.get('class', []) or 'detailed holdings and analytics' in a.text.lower():
        holdings_link = a['href']
        break

if not holdings_link:
    print("Holdings CSV link not found. Saving product page to 'product_page.html'.")
    with open("product_page.html", "w", encoding="utf-8") as f:
        f.write(response.text)


In [ ]:
# Ensure the link is absolute
if not holdings_link.startswith('http'):
    holdings_link = "https://www.ishares.com" + holdings_link
print(f"Found holdings link: {holdings_link}")

# Step 3: Fetch the CSV file
file_response = requests.get(holdings_link, headers=headers)
if file_response.status_code != 200:
    print(f"Failed to download CSV file. Status code: {file_response.status_code}")

In [ ]:
# Step 4: Parse the CSV
try:
    # Extract Shares Outstanding from preamble using csv reader
    preamble = file_response.text.splitlines()[:8]
    shares_outstanding = None
    for line in csv.reader(preamble):
        if line and "Shares Outstanding" in line[0]:
            shares_outstanding = line[1].strip('"')
            break
    if shares_outstanding:
        print(f"Shares Outstanding: {shares_outstanding}")
    else:
        print("Shares Outstanding not found in CSV preamble.")

    # Parse holdings table, skipping 9 rows to start at headers
    df = pd.read_csv(io.StringIO(file_response.text), skiprows=9)
    
    # Extract BTC Held from Quantity column
    btc_row = df[df['Ticker'] == 'BTC']
    if not btc_row.empty:
        btc_held = btc_row['Quantity'].values[0]
        print(f"BTC Held: {btc_held}")
    else:
        print("BTC not found in holdings table.")

except Exception as e:
    print(f"Failed to parse CSV: {e}")
    with open("holdings_response.csv", "w", encoding="utf-8") as f:
        f.write(file_response.text)
    print("Saved CSV to 'holdings_response.csv' for inspection.")

To get total BTC held:
1. Use the Alpha Vantage API's `ETF_PROFILE` function on `IBIT`
2. Extract `net_assets`
3. Get the current `BTC/USD` price
4. Total `BTC` is `net_assets`/`BTC/USD` price